In [47]:
from json import dumps
from os.path import abspath
from re import sub

# Splits each line (a path representation) in the text file into a list of path elements.
# We can use the length of the list to determine the nesting depth of each OU
org_ou_file = './tests/organization_ous.txt'
org_ou_paths = []
with open(abspath(org_ou_file)) as f:
    org_ou_paths = f.readlines()
org_ou_path_list = [sub('^\/', '', ou_path).rstrip('\n').split('/') for ou_path in org_ou_paths]
print(dumps(org_ou_path_list, indent=4))

[
    [
        "Cool BU"
    ],
    [
        "Cool BU",
        "Great Cloud Service",
        "Production"
    ],
    [
        "Cool BU",
        "Great Cloud Service",
        "Staging"
    ],
    [
        "Cool BU",
        "Great Cloud Service",
        "Development"
    ],
    [
        "Cool BU",
        "Another Great Service",
        "Production"
    ],
    [
        "Cool BU",
        "Another Great Service",
        "Staging"
    ],
    [
        "Cool BU",
        "Another Great Service",
        "Development"
    ],
    [
        "Cool BU",
        "Cool Admins",
        "Control",
        "Prod",
        ""
    ],
    [
        "Cool BU",
        "Cool Admins",
        "Control",
        "Stage",
        ""
    ],
    [
        "Cool BU",
        "Cool Admins",
        "Control",
        "Dev",
        ""
    ],
    [
        "Ecstatic BU"
    ],
    [
        "Ecstatic BU",
        "Oracle Admins",
        "DB1",
        "Prd"
    ],
    [
        "Ecstatic BU",
    

In [34]:
# {
#   node1 = [child1, child2, child3],
#   node2 = [child1, child2, child3],
#   note3 = [child1, child2, child3]
# }

# - create a UUID for each node. since we can't guarantee path uniqueness due to mutability,
#   the UUID can be used to track notes prior to getting an ou_id from the Organizations API.
# - paths with single elements are root OUs
# - paths with more than one element are nested OUs
# - create the OUs starting with the roots inwards
# - 

In [50]:
def get_root_ous(ou_path_list: list[str]):
    if len(ou_path_list) == 0:
        raise Exception('Received an empty list')
    root_ous = []
    for ou_path in ou_path_list:
        if len(ou_path) == 0:
            print('Warning: OU path element is empty. Skipping.')
            continue
        root_ous.append(ou_path[0])
    return list(set(root_ous))

In [51]:
root_ous = get_root_ous(org_ou_path_list)
print(dumps(root_ous))

["Cool BU", "Disaster Prep BU", "Ecstatic BU"]
